<h1>Import Libraries</h1>

In [1]:
# import altair with an abbreviated alias
import altair as alt
import csv
import pandas as pd

<h1>Review Riverside Commute Data</h1>

In [2]:
df_walk_riverside = pd.read_csv("../results/commute_riverside_stats.csv", index_col=[0])

In [3]:
alt.Chart(df_walk_riverside).mark_circle(size=30).encode(
    x='distance_sum',
    y='elev',
    tooltip=['lat', 'long', 'slope_angle']
).interactive()

alt.Chart(...)

<h3>Observation 1</h3>

<br>
“false” valley around 200m.<br>
plausible explanation is that GPS lookup looks at ground level and misses bridge over Trinity Cemetery<br>
start: 40.83404 -73.9493<br>
end: 40.83398 -73.94934<br>
155th at Riverside<br>
        
<img src="../images/riverside_155_satellite.png" width=50%>

<h3>Observation 2</h3>
“false” valley runs from around 1,800 to 2,500<br>
plausible explanation is that Strava and AllTrails are looking up elevation based on GPS at ground level, not bridge level<br>
<br>
start: 40.82169N 73.95675W <a href="https://www.google.com/maps/place/40%C2%B049'18.1%22N+73%C2%B057'24.3%22W/@40.821694,-73.9593249,17z/data=!3m1!4b1!4m4!3m3!8m2!3d40.82169!4d-73.95675?entry=ttu">136th and Riverside</a><br>
<br>
end: 40.81676N 73.96131W<br>
<a href="https://www.google.com/maps/place/40%C2%B049'18.1%22N+73%C2%B057'24.3%22W/@40.821694,-73.9593249,17z/data=!3m1!4b1!4m4!3m3!8m2!3d40.82169!4d-73.95675?entry=ttu">129th and Riverside</a><br>

<img src="../images/riverside_125_satellite.png" width=50%>

<h1>Fix Riverside Elevation Data</h1>

In [80]:
df_walk_riverside = pd.read_csv("../data/commute_riverside.csv", index_col=[0])

<h3>Remove First Bridge</h3>
   <strong>
    <ol>
    <li>Find GPS coordinates for 155th (over Trinity Cemetery)
        <li>Drop GPS coordinates for this section
    </ul>
    </strong>

In [81]:
gps_bridge1_start = df_walk_riverside.loc[df_walk_riverside['lat'] == 40.83404].index[0]
gps_bridge1_start

11

In [82]:
gps_bridge1_end = df_walk_riverside.loc[df_walk_riverside['lat'] == 40.83398].index[0]
gps_bridge1_end

14

In [83]:
df_walk_riverside = df_walk_riverside.drop(df_walk_riverside.index[gps_bridge1_start:gps_bridge1_end+1], axis=0)
df_walk_riverside = df_walk_riverside.reset_index()
df_walk_riverside = df_walk_riverside.drop(columns=['index'])

,lat,long,elev
0,40.83471,-73.94743,26.10
1,40.83472,-73.94747,26.01
2,40.83444,-73.94766,26.27
3,40.83432,-73.94775,26.00
4,40.83413,-73.94808,23.63
...,...,...,...
146,40.80850,-73.96473,36.96
147,40.80820,-73.96403,41.00
148,40.80818,-73.96404,41.00
149,40.80812,-73.96409,40.58


<h3>Step 3 - Surgically Replace Data for 2nd bridge</h3>
2nd bridge: between 136th and 129th (over 12th Avenue)- drop data and replace with Google Earth readings
Append altitude data on Riverside bridge manually looked up for each block between 129th and 136th on <a href="https://earth.google.com/web/@40.81864262,-73.95820922,7.25365032a,400.670882d,35y,0.88414262h,0t,0r">Google Earth</a>

In [84]:
gps_bridge2_start = df_walk_riverside.loc[df_walk_riverside['lat'] == 40.82169].index[0]
gps_bridge2_start

64

In [85]:
gps_bridge2_end = df_walk_riverside.loc[df_walk_riverside['lat'] == 40.81676].index[0]
gps_bridge2_end

89

In [86]:
gps_end = len(df_walk_riverside)
gps_end

151

In [87]:
df_walk_riverside_start = df_walk_riverside[0:gps_bridge2_start]
df_walk_riverside_start

,lat,long,elev
0,40.83471,-73.94743,26.10
1,40.83472,-73.94747,26.01
2,40.83444,-73.94766,26.27
3,40.83432,-73.94775,26.00
4,40.83413,-73.94808,23.63
...,...,...,...
59,40.82280,-73.95572,19.00
60,40.82274,-73.95575,19.37
61,40.82250,-73.95589,20.00
62,40.82229,-73.95607,21.67


In [88]:
df_walk_riverside_end = df_walk_riverside[gps_bridge2_end+1:gps_end]
df_walk_riverside_end

,lat,long,elev
90,40.81672,-73.96136,29.00
91,40.81669,-73.96138,29.00
92,40.81664,-73.96143,28.68
93,40.81658,-73.96150,28.75
94,40.81653,-73.96156,28.56
...,...,...,...
146,40.80850,-73.96473,36.96
147,40.80820,-73.96403,41.00
148,40.80818,-73.96404,41.00
149,40.80812,-73.96409,40.58


<h3>Adjustment 2</h3> Append altitude data on Riverside bridge manually looked up for each block between 129th and 136th on <a href="https://earth.google.com/web/@40.81864262,-73.95820922,7.25365032a,400.670882d,35y,0.88414262h,0t,0r">Google Earth</a>

In [89]:
#append
df_walk_bridge = pd.read_csv("../data/commute_riverside_bridge_136.csv")
df_walk_riverside_cleaned = pd.concat([df_walk_riverside_start,df_walk_bridge,df_walk_riverside_end])

,lat,long,elev,cross_street,lat_long_google_maps
0,40.83471,-73.94743,26.10,NaN,NaN
1,40.83472,-73.94747,26.01,NaN,NaN
2,40.83444,-73.94766,26.27,NaN,NaN
3,40.83432,-73.94775,26.00,NaN,NaN
4,40.83413,-73.94808,23.63,NaN,NaN
...,...,...,...,...,...
146,40.80850,-73.96473,36.96,NaN,NaN
147,40.80820,-73.96403,41.00,NaN,NaN
148,40.80818,-73.96404,41.00,NaN,NaN
149,40.80812,-73.96409,40.58,NaN,NaN


In [90]:
#sort, re-index, drop extra columns
df_walk_riverside_cleaned = (df_walk_riverside_cleaned
    .reset_index()
    .drop(columns =["index","cross_street","lat_long_google_maps"])
)
df_walk_riverside_cleaned

,lat,long,elev
0,40.83471,-73.94743,26.10
1,40.83472,-73.94747,26.01
2,40.83444,-73.94766,26.27
3,40.83432,-73.94775,26.00
4,40.83413,-73.94808,23.63
...,...,...,...
128,40.80850,-73.96473,36.96
129,40.80820,-73.96403,41.00
130,40.80818,-73.96404,41.00
131,40.80812,-73.96409,40.58


In [91]:
df_walk_riverside_cleaned.to_csv("../data/commute_riverside_cleaned.csv")

<h1>Go Back to Step 2 to recalculate Riverside stats, then on to Step 5</h1>